In [1]:
import copy


import sys
import os
import scipy
import tensorflow as tf
from keras.layers import Input, Conv2D, Lambda,  Flatten, Dropout,Reshape,UpSampling2D
from keras.models import Model, Sequential
from keras.regularizers import l2,l1_l2,l1
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, LearningRateScheduler
from keras.backend.tensorflow_backend import set_session
from keras.losses import mean_squared_error,cosine_proximity
from keras.utils import Sequence
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [2]:
# !pip install -q git+https://github.com/tensorflow/docs

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


# Data- Handler

In [4]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn import preprocessing


class Data_handler():
    """Generate batches for FMRI prediction
    frames_back - how many video frames to take before FMRI frame
    frames_forward - how many video frames to take after FMRI frame
    """

    def __init__(self,matlab_file,test_img_csv ='./images/image_test_id.csv',train_img_csv ='./images/image_training_id.csv',voxel_spacing =3,log = 0):
        mat = loadmat(matlab_file)
        self.data = mat['dataSet'][:,3:]
        self.sample_meta = mat['dataSet'][:,:3]
        meta=mat['metaData']


        self.meta_keys = list(l[0] for l in meta[0][0][0][0])
        self.meta_desc = list(l[0] for l in meta[0][0][1][0])
        self.voxel_meta = np.nan_to_num(meta[0][0][2][:,3:])
        test_img_df = pd.read_csv(test_img_csv, header=None)
        train_img_df =pd.read_csv(train_img_csv, header=None)
        self.test_img_id = test_img_df[0].values
        self.train_img_id = train_img_df[0].values
        self.sample_type = {'train':1 , 'test':2 , 'test_imagine' : 3}
        self.voxel_spacing = voxel_spacing

        self.log = log

    def get_meta_field(self,field = 'DataType'):
        index = self.meta_keys.index(field)
        if(index <3): # 3 first keys are sample meta
            return self.sample_meta[:,index]
        else:
            return self.voxel_meta[index]


    def print_meta_desc(self):
        print(self.meta_desc)

    def get_labels(self, imag_data = 0,test_run_list = None):
#         le = preprocessing.LabelEncoder()

        img_ids = self.get_meta_field('Label')
        type = self.get_meta_field('DataType')
        train = (type == self.sample_type['train'])
        test = (type == self.sample_type['test'])
        imag = (type == self.sample_type['test_imagine'])

        img_ids_train = img_ids[train]
        img_ids_test = img_ids[test]
        img_ids_imag = img_ids[imag]


        train_labels  = []
        test_labels  =  []
        imag_labels = []
        for id in img_ids_test:
            idx = (np.abs(id - self.test_img_id)).argmin()
            test_labels.append(idx)

        for id in img_ids_train:
            idx = (np.abs(id - self.train_img_id)).argmin()
            train_labels.append(idx)

        for id in img_ids_imag:
            idx = (np.abs(id - self.test_img_id)).argmin()
            imag_labels.append(idx)

        if (test_run_list is not None):
            run = self.get_meta_field('Run')
            test = (self.get_meta_field('DataType') == 2).astype(bool)
            run = run[test]

            select = np.in1d(run, test_run_list)
            test_labels = test_labels[select]

        #imag_labels = le.fit_transform(img_ids_imag)
        if(imag_data):
            return np.array(train_labels), np.array(test_labels), np.array(imag_labels)
        else:
            return np.array(train_labels),np.array(test_labels)





    def get_data(self,normalize =1 ,roi = 'ROI_VC',imag_data = 0,test_run_list = None):   # normalize 0-no, 1- per run , 2- train/test seperatly
        type = self.get_meta_field('DataType')
        train = (type == self.sample_type['train'])
        test = (type == self.sample_type['test'])
        test_imag = (type == self.sample_type['test_imagine'])
        test_all  = np.logical_or(test,test_imag)

        roi_select = self.get_meta_field(roi).astype(bool)
        data = self.data[:,roi_select]

        if(self.log ==1):
            data = np.log(1+np.abs(data))*np.sign(data)


        if(normalize==1):

            run = self.get_meta_field('Run').astype('int')-1
            num_runs = np.max(run)+1
            data_norm = np.zeros(data.shape)

            for r in range(num_runs):
                data_norm[r==run] = sklearn.preprocessing.scale(data[r==run])
            train_data = data_norm[train]
            test_data  = data_norm[test]
            test_all = data_norm[test_all]
            test_imag = data_norm[test_imag]

        else:
            train_data = data[train]
            test_data  =  data[test]
            if(normalize==2):
                train_data = sklearn.preprocessing.scale(train_data)
                test_data = sklearn.preprocessing.scale(test_data)


        if(self.log ==2):
            train_data = np.log(1+np.abs(train_data))*np.sign(train_data)
            test_data = np.log(1+np.abs(test_data))*np.sign(test_data)
            train_data = sklearn.preprocessing.scale(train_data)
            test_data = sklearn.preprocessing.scale(test_data)



        test_labels =  self.get_labels()[1]
        imag_labels = self.get_labels(1)[2]
        num_labels = max(test_labels)+1
        test_data_avg = np.zeros([num_labels,test_data.shape[1]])
        test_imag_avg = np.zeros([num_labels,test_data.shape[1]])

        if(test_run_list is not None):
            run = self.get_meta_field('Run')
            test = (self.get_meta_field('DataType') == 2).astype(bool)
            run = run[test]

            select = np.in1d(run, test_run_list)
            test_data = test_data[select,:]
            test_labels = test_labels[select]

        for i in range(num_labels):
            test_data_avg[i] = np.mean(test_data[test_labels==i],axis=0)
            test_imag_avg[i] = np.mean(test_imag[imag_labels == i], axis=0)
        if(imag_data):
            return train_data, test_data, test_data_avg,test_imag,test_imag_avg

        else:
            return train_data, test_data, test_data_avg

    def get_voxel_loc(self):
        x = self.get_meta_field('voxel_x')
        y = self.get_meta_field('voxel_y')
        z = self.get_meta_field('voxel_z')
        dim = [int(x.max() -x.min()+1),int(y.max() -y.min()+1), int(z.max() -z.min()+1)]
        return [x,y,z] , dim

# Loading Data

In [5]:
handler=Data_handler(matlab_file ='./Subject3.mat')

In [6]:
Y,Y_test,Y_test_avg = handler.get_data(roi = 'ROI_VC')
labels_train, labels = handler.get_labels()
NUM_VOXELS = Y.shape[1]

In [7]:
NUM_VOXELS

4643

In [8]:
file= np.load('images_npz.npz')
X = file['train_images']
X_test = file['test_images']

X= X[labels_train]
X_test_sorted = X_test
X_test = X_test[labels]

# Losses and Schedules

In [9]:
initial_lrate = 0.1


def step_decay(epoch):
    
    lrate = 0.1 #Learning Rate
    
    # So for various epochs we will be using various learning rates
    
    if(epoch>20):
        lrate = 0.01
    if (epoch > 35):
        lrate = 0.001
    if (epoch > 45):
        lrate = 0.0001
    if (epoch > 50):
        lrate = 0.00001

    return lrate

def combined_loss(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)+ 0.1*cosine_proximity(y_true, y_pred)

# Regularizer

In [10]:
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf
import numpy as np
from keras.regularizers import Regularizer
from keras.regularizers import l2,l1_l2,l1

class GroupLasso(Regularizer):
    """ GroupLasso Regularizer
    Used for regularization of fc layer, from conv activations to dense activations
    Assumes that input weight x is formatted the following way:
    height, width, channels, output units
    channels in the same position will be grouped together (for each output unit separately)
     Arguments
        l1: Float; L1 regularization factor.
        gl: Float; gl group regularization factor.
        gl_n: first order neighbor's contribution to group regularization
    """

    def __init__(self, l1=0., gl=0.,gl_n=0):
        self.l1 = K.cast_to_floatx(l1)
        self.gl = K.cast_to_floatx(gl)
        self.gl_n = K.cast_to_floatx(gl_n)


    def __call__(self, x):

        regularization = self.l1 * K.mean(K.abs(x))
        x_sq = K.square(x)

        if (self.gl_n > 0):
            x_sq_pad = tf.pad(x_sq, [[1, 1], [1, 1], [0, 0], [0, 0]], "SYMMETRIC")
            x_sq_avg_n = (x_sq_pad[:-2, 1:-1] + x_sq_pad[2:, 1:-1] + x_sq_pad[1:-1, :-2] + x_sq_pad[1:-1, 2:]) / 4
            x_sq = (x_sq + self.gl_n * x_sq_avg_n) / (1 + self.gl_n)
        regularization += self.gl * K.mean(K.sqrt(K.mean(x_sq, axis=-2)))  # assumes weight structure x,y,ch,voxel
        return regularization


    def get_config(self):
        return {'l1': float(self.l1), 'l2': float(self.gl)}


# Model

In [11]:
MEAN_PIXELS = [123.68, 116.779, 103.939]

In [12]:
def subtract_mean(x):
    mean = tf.constant(MEAN_PIXELS,shape=[1,1,1,3],dtype=tf.float32)
    return tf.subtract(x,mean)

In [13]:
def _weights(net_layers, layer, expected_layer_name):
    """ Return the weights and biases trained by VGG
    """
    W = net_layers[0][layer][0][0][2][0][0]
    b = net_layers[0][layer][0][0][2][0][1]
    layer_name = net_layers[0][layer][0][0][0][0]
    assert layer_name == expected_layer_name
    return W, b.reshape(b.size)

In [14]:
def list_prod(l):
    prod = 1
    for e in l:
        prod*=e
    return prod

In [15]:
def conv2d_relu_(prev_layer,net_layers, layer, layer_name,stride=1, pad=None):
    """ Return the Conv2D layer with RELU using the weights, biases from the VGG
    model at 'layer'.
    Inputs:
        net_layers: holding all the layers of VGGNet
        prev_layer: the output tensor from the previous layer
        layer: the index to current layer in net_layers
        layer_name: the string that is the name of the current layer.
                    It's used to specify variable_scope.
    Output:
        relu applied on the convolution.
    """
    W, b = _weights(net_layers, layer, layer_name)
    W = tf.constant(W, name='weights')
    b = tf.constant(b, name='bias')

    conv2d = tf.nn.conv2d(prev_layer,W, strides=[1, stride, stride, 1], padding='SAME')
    return tf.nn.relu(conv2d + b)


In [16]:
class dense_c2f_gl(Layer):

    def __init__(self, units=1024,l1=0.1,gl=0.1,gl_n=0,kernel_init = "glorot_normal", **kwargs):
        self.units = units
        self.l1 = l1
        self.gl = gl
        self.gl_n = gl_n
        self.kernel_init =kernel_init
        super(dense_c2f_gl, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.s = input_shape
        shape = list(input_shape[1:])+[self.units]
        self.kernel = self.add_weight(name='dense_c2f_gl_kernel',
                                      shape=shape,
                                      regularizer= GroupLasso(l1=self.l1,gl=self.gl,gl_n= self.gl_n) ,

                                      initializer=self.kernel_init,
                                      trainable=True)
        self.bias = self.add_weight(name='dense_c2f_gl_bias', shape=(self.units,),initializer="glorot_normal",trainable=True)

        super(dense_c2f_gl, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        input_shape = self.s
        size =  list_prod(input_shape[1:])
        x = tf.reshape(x, [-1, size])
        w = tf.reshape(self.kernel,[size ,self.units])
        output = K.dot(x, w)

        output = K.bias_add(output, self.bias)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [17]:
class encoder_param():
    def __init__(self,num_voxels):
        self.num_voxels = num_voxels
        self.resolution = 112
        self.conv_l1_reg = 1e-5
        self.conv_l2_reg = 0.001
        self.fc_reg_l1 = 10
        self.fc_reg_gl = 800
        self.fc_reg_gl_n = 0.5
        self.conv_ch = 32
        self.num_conv_layers = 2
        self.conv1_stride = 2
        self.conv_last_dim  =14
        self.drop_out = 0.5

        self.caffenet_models_weights=scipy.io.loadmat('imagenet-caffe-ref.mat') #Pretrained Alexnet network weights
        self.caffenet_models_weights=self.caffenet_models_weights['layers']

In [18]:
def encoder(param,name = 'encoder'):
    input_shape = (param.resolution, param.resolution, 3)
    model = Sequential(name =name)
    model.add(Lambda(lambda img: img[:, :, :, ::-1] * 255.0, input_shape=input_shape))
    model.add(Lambda(subtract_mean))
    model.add(Lambda(conv2d_relu_,
                            arguments={'net_layers': param.caffenet_models_weights, 'layer': 0, 'layer_name': 'conv1', 'stride': param.conv1_stride,
                                       'pad': 'SAME'}))
    model.add(BatchNormalization(axis=-1))
    for i in range(param.num_conv_layers):
        model.add(Conv2D(param.conv_ch, (3, 3), padding='same', kernel_initializer="glorot_normal", activation='relu',
                                kernel_regularizer=l1_l2(param.conv_l1_reg,param.conv_l2_reg), strides=(2, 2)))

        model.add(BatchNormalization(axis=-1))

    model.add(Flatten())  # flatten needed for dropout, without it suboptimal results are observed
    model.add(Dropout(param.drop_out))

    model.add(Reshape((param.conv_last_dim, param.conv_last_dim, param.conv_ch)))
    model.add(dense_c2f_gl(units=param.num_voxels, l1=param.fc_reg_l1, gl=param.fc_reg_gl, gl_n=param.fc_reg_gl_n))
    return model

In [19]:
enc_param = encoder_param(NUM_VOXELS)
enc_param.drop_out = 0.5

vision_model = encoder(enc_param)
vision_model.compile(loss=combined_loss, optimizer= SGD(lr=initial_lrate,decay = 0.0 , momentum = 0.9,nesterov=True),metrics=['mse','cosine_proximity','mae'])
print(vision_model.summary())

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 112, 112, 3)       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 112, 112, 3)       0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 56, 56, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        27680     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248

# CallBacks

In [20]:
# callbacks = []
# log_path='./logs'

# # log_path = config_file.encoder_tenosrboard_logs
# tb_callback = TensorBoard(log_path)
# tb_callback.set_model(vision_model)
# callbacks.append(tb_callback)
callbacks=[]
reduce_lr = LearningRateScheduler(step_decay)
callbacks.append(reduce_lr)

# Train and Save the weights

In [21]:
from scipy.ndimage import shift
def rand_shift(img,max_shift = 0 ):
    """
    randomly shifted image
    :param img: image
    :param max_shift: image output size
    :return randomly shifted image
    """
    x_shift, y_shift = np.random.randint(-max_shift, max_shift + 1, size=2)
    img_shifted = shift(img,[x_shift, y_shift, 0], prefilter=False, order=0, mode='nearest')
    return img_shifted

In [22]:
class batch_generator_dec(Sequence):

    """
    Generates batches for decoder model
    :param X: images
    :param Y: fMRI activations
    (X,Y correspond )
    :param batch_size: batch_size
    """

    def __init__(self,  X, Y, batch_size =32):
        self.indexes = {}
        self.batch_size = batch_size
        self.Y = Y
        self.X = X
        self.indexes  = np.random.permutation(self.Y.shape[0])

    def __len__(self):
        'Denotes the number of batches per epoch'
        return max(int(self.Y.shape[0] // self.batch_size), 1) #
    def __getitem__(self,batch_num):
        indexes = (self.indexes)[batch_num * self.batch_size:(batch_num + 1) * self.batch_size]
        return self.Y[indexes], self.X[indexes]

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [23]:



class batch_generator_enc(batch_generator_dec):
    """
    Generates batches for encoder model
    :param X: images
    :param Y: fMRI activations
    (X,Y correspond )
    :param batch_size: batch_size
    :param max_shift: max random shift applied on images
    """

    def __init__(self, X, Y, batch_size=32,max_shift = 5):
        super().__init__(X, Y, batch_size)
        self.max_shift = max_shift

    def __getitem__(self,batch_num):
        y, x = super().__getitem__(batch_num)
        x_shifted = np.zeros(x.shape)
        for i in range(x.shape[0]):
            x_shifted[i] = rand_shift(x[i],self.max_shift)
        return x_shifted,y

In [24]:
train_generator = batch_generator_enc(X, Y, batch_size=64,max_shift = 5)
test_generator = batch_generator_enc(X_test_sorted, Y_test_avg, batch_size=50,max_shift = 0)

In [25]:
h=vision_model.fit_generator(train_generator,epochs=80,validation_data=test_generator,verbose=2,use_multiprocessing=False,callbacks=callbacks) #, steps_per_epoch=1200//64 , validation_steps=1

Epoch 1/80
 - 87s - loss: 2.1823 - mse: 0.9939 - cosine_proximity: 0.0368 - mae: 0.7874 - val_loss: 1.1972 - val_mse: 0.1636 - val_cosine_proximity: 0.1318 - val_mae: 0.2991
Epoch 2/80
 - 78s - loss: 1.8213 - mse: 0.9460 - cosine_proximity: 0.1669 - mae: 0.7672 - val_loss: 0.8290 - val_mse: 0.1261 - val_cosine_proximity: 0.1539 - val_mae: 0.2657
Epoch 3/80
 - 74s - loss: 1.4765 - mse: 0.9219 - cosine_proximity: 0.2345 - mae: 0.7571 - val_loss: 0.5042 - val_mse: 0.0899 - val_cosine_proximity: 0.2165 - val_mae: 0.2257
Epoch 4/80
 - 69s - loss: 1.2464 - mse: 0.9219 - cosine_proximity: 0.2541 - mae: 0.7572 - val_loss: 0.3407 - val_mse: 0.0844 - val_cosine_proximity: 0.2444 - val_mae: 0.2193
Epoch 5/80
 - 70s - loss: 1.1637 - mse: 0.9308 - cosine_proximity: 0.2175 - mae: 0.7610 - val_loss: 0.2812 - val_mse: 0.0791 - val_cosine_proximity: 0.2873 - val_mae: 0.2125
Epoch 6/80
 - 69s - loss: 1.1382 - mse: 0.9345 - cosine_proximity: 0.2025 - mae: 0.7625 - val_loss: 0.2612 - val_mse: 0.0780 - val

Epoch 48/80
 - 142s - loss: 0.9632 - mse: 0.9228 - cosine_proximity: 0.2541 - mae: 0.7572 - val_loss: 0.0812 - val_mse: 0.0638 - val_cosine_proximity: 0.4838 - val_mae: 0.1936
Epoch 49/80
 - 113s - loss: 0.9633 - mse: 0.9230 - cosine_proximity: 0.2535 - mae: 0.7572 - val_loss: 0.0810 - val_mse: 0.0638 - val_cosine_proximity: 0.4840 - val_mae: 0.1936
Epoch 50/80
 - 123s - loss: 0.9582 - mse: 0.9182 - cosine_proximity: 0.2551 - mae: 0.7554 - val_loss: 0.0808 - val_mse: 0.0638 - val_cosine_proximity: 0.4843 - val_mae: 0.1935
Epoch 51/80
 - 107s - loss: 0.9618 - mse: 0.9218 - cosine_proximity: 0.2537 - mae: 0.7569 - val_loss: 0.0806 - val_mse: 0.0637 - val_cosine_proximity: 0.4844 - val_mae: 0.1935
Epoch 52/80
 - 107s - loss: 0.9591 - mse: 0.9191 - cosine_proximity: 0.2541 - mae: 0.7560 - val_loss: 0.0806 - val_mse: 0.0637 - val_cosine_proximity: 0.4844 - val_mae: 0.1935
Epoch 53/80
 - 108s - loss: 0.9629 - mse: 0.9230 - cosine_proximity: 0.2537 - mae: 0.7573 - val_loss: 0.0806 - val_mse: 

In [26]:
# vision_model.save_weights('./encoder_weights')

In [27]:

# list all data in history
print(h.history.keys())

dict_keys(['val_loss', 'val_mse', 'val_cosine_proximity', 'val_mae', 'loss', 'mse', 'cosine_proximity', 'mae', 'lr'])


In [28]:
import matplotlib.pyplot as plt
plt.plot(h.history['mse'])
plt.plot(h.history['val_mse'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


import matplotlib.pyplot as plt
plt.plot(h.history['mae'])
plt.plot(h.history['val_mae'])
plt.title('model mae')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(h.history['cosine_proximity'])
plt.plot(h.history['val_cosine_proximity'])
plt.title('model cosine proximity')
plt.ylabel('cosine_promximity')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# # summarize history for loss
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>